In [1]:
# Checking out different shots for Shor's condesed run 


In [2]:
# Defining the libraries 

import boto3
from braket.circuits import Circuit, circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
import math
from fractions import Fraction

In [3]:
def inverse_qft(qubits):
    # instantiate circuit object
    qftcirc = Circuit()
    
    # get number of qubits
    num_qubits = len(qubits)
    
    # First add SWAP gates to reverse the order of the qubits:
    for i in range(math.floor(num_qubits/2)):
        qftcirc.swap(qubits[i], qubits[-i-1])
        
    # Start on the last qubit and work to the first.
    for k in reversed(range(num_qubits)):
    
        # Apply the controlled rotations, with weights (angles) defined by the distance to the control qubit.
        # These angles are the negative of the angle used in the QFT.
        # Start on the last qubit and iterate until the qubit after k.  
        # When num_qubits==1, this loop does not run.
        for j in reversed(range(1, num_qubits - k)):
            angle = -2*math.pi/(2**(j+1))
            qftcirc.cphaseshift(qubits[k+j],qubits[k], angle)
            
        # Then add a Hadamard gate
        qftcirc.h(qubits[k])
    
    return qftcirc

def CreatePossible_Period_Values(counts):
    # Currently there is hard coding , change it later
    set_a = set()
    for measurement in counts.keys():
        decimal = int(measurement, 2)/2**3
        r2 = (Fraction(decimal,).limit_denominator())
        value = r2.denominator
        set_a.add(value)
        list_period = []
    for ele in set_a:
        list_period.append(ele)
    list_period.sort()
    return list_period
def findFactors(list_period):
    # Cuurently hard coded change later
    factors = set()
    for period_possible in list_period:
        #Check if even
        if(period_possible %2 == 0):
            val = period_possible/2
            val_1 = 13**val
            val_2 = val_1+1
            val_3 = val_1-1

            val_4 = math.gcd(35,int(val_2))
            val_5 = math.gcd(35,int(val_3))
            factors.add(val_4)
            factors.add(val_5)
        else:
            continue

    factors_list = []
    for factor in factors:
        if(factor != 1 and factor != 35):
            factors_list.append(factor)
    return factors_list

In [4]:
#Creating the circuit for 13^i mod(35)
circuit = Circuit()

circuit.h(0)
circuit.h(1)
circuit.h(2)

circuit.ccnot(0,3,4)
circuit.cnot(0,3)

circuit.ccnot(1,3,4)
circuit.cnot(1,3)
circuit.ccnot(1,3,4)
circuit.cnot(1,3)
circuit.ccnot(1,3,4)
circuit.cnot(1,3)

circuit.ccnot(2,3,4)
circuit.cnot(2,3)
circuit.ccnot(2,3,4)
circuit.cnot(2,3)
circuit.ccnot(2,3,4)
circuit.cnot(2,3)
circuit.ccnot(2,3,4)
circuit.cnot(2,3)


circuit.add_circuit(inverse_qft(range(3)))

print(circuit)

T  : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16| 17 |18|     19     |      20      |     21     |22|
                                                                                                   
q0 : -H-C-C------------------------------------SWAP-------------------PHASE(-0.79)-PHASE(-1.57)-H--
        | |                                    |                      |            |               
q1 : -H-|-|-C-C-C-C-C-C------------------------|-------PHASE(-1.57)-H-|------------C---------------
        | | | | | | | |                        |       |              |                            
q2 : -H-|-|-|-|-|-|-|-|-C-C--C--C--C--C--C--C--SWAP-H--C--------------C----------------------------
        | | | | | | | | | |  |  |  |  |  |  |                                                      
q3 : ---C-X-C-X-C-X-C-X-C-X--C--X--C--X--C--X------------------------------------------------------
        |   |   |   |   |    |     |     |                                                         


In [5]:
# get the account ID
aws_account_id = boto3.client("sts").get_caller_identity()["Account"]
# the name of the bucket
my_bucket = "DN1-bucket"
# the name of the folder in the bucket
my_prefix = "simulation-output"
s3_folder = (my_bucket, my_prefix)

In [7]:
# choose the cloud-based on-demand simulator to run your circuit
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

In [8]:
result = device.run(circuit, shots=1000).result()
counts = result.measurement_counts


In [9]:
for measurement in counts.keys():
    decimal = int(measurement, 2)/2**3

In [10]:
# Getting possible period values 
period_Possiblity = CreatePossible_Period_Values(counts)
print(period_Possiblity)

[1, 2, 4, 8]


In [ ]:
factors = findFactors(period_Possiblity)
print(factors)